# Stock data  preprocessing , labelling  and mapping 
# The following codes were applied on all 5 companies (1 company per run)  and then saved output for each company was saved in folder (Labeled_Data_modeling):
## By: Hadeel Sameh

In [1]:
import pandas as pd
df=pd.read_csv('Fainancial_data/HistoricalData_IBM.csv')

In [2]:
df.head()

,Date,Close/Last,Volume,Open,High,Low
0,10/06/2021,$142.36,5328433,$142.48,$143.37,$140.89
1,10/05/2021,$143.15,6976648,$144.75,$145,$142.64
2,10/04/2021,$144.11,7351128,$142.74,$146,$142.3501
3,10/01/2021,$143.32,6604064,$141,$143.97,$140.37
4,09/30/2021,$138.93,5824431,$140,$140.57,$138.5


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 128 entries, 0 to 127
Data columns (total 6 columns):
Date          128 non-null object
Close/Last    128 non-null object
Volume        128 non-null int64
Open          128 non-null object
High          128 non-null object
Low           128 non-null object
dtypes: int64(1), object(5)
memory usage: 6.1+ KB


## Inserting Weekend days:

In [4]:
df.set_index('Date',drop=True,inplace=True)
df.index = pd.DatetimeIndex(df.index)
d = pd.date_range(df.index.min(), df.index.max(), freq='D')
df = df.reindex(d)
df['Date'] = pd.DatetimeIndex(df.index)

In [5]:
df.reset_index(drop=True, inplace=True)

In [6]:
a=[]
a=df.columns
for i in a:
    print(f'does column {i} has null values? {df[i].isnull().any()}')


does column Close/Last has null values? True
does column Volume has null values? True
does column Open has null values? True
does column High has null values? True
does column Low has null values? True
does column Date has null values? False


## removing dollar sign and coverting columns to numerical:

In [7]:
#make date column the first
first_column = df.pop('Date')
df.insert(0, 'Date', first_column)

In [8]:
string_col=['Date','Close/Last','Open','High','Low']
for col in string_col:
    df[col] = df[col].map(lambda x: str(x).replace('$',''))

In [9]:
string_2_float=['Close/Last','Open','High','Low']
for col in string_2_float:
    df[col] = df[col].apply(float)

## filling null values :

In [10]:
import numpy as np
rows=np.arange(3,182) 
null_col=['High','Close/Last','Open','Low','Volume']
for i in null_col:
    for k in rows:
        df[i] = df[i].fillna(np.mean((df[i][k-1]+df[i][k+1])/2))
        

### the result:

In [11]:
df['Date'] = pd.to_datetime(df['Date']).dt.date
df.head()

,Date,Close/Last,Volume,Open,High,Low
0,2021-04-07,134.93,2976136.0,133.8400,134.9400,133.780
1,2021-04-08,135.12,4087228.0,134.5700,135.6299,134.160
2,2021-04-09,135.73,3023916.0,134.8700,135.7400,134.710
3,2021-04-10,133.61,4811004.0,133.1625,134.0750,132.185
4,2021-04-11,133.61,4811004.0,133.1625,134.0750,132.185


In [12]:
for i in a:
    print(f'does column {i} has null values? {df[i].isnull().any()}')

does column Close/Last has null values? False
does column Volume has null values? False
does column Open has null values? False
does column High has null values? False
does column Low has null values? False
does column Date has null values? False


In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 183 entries, 0 to 182
Data columns (total 6 columns):
Date          183 non-null object
Close/Last    183 non-null float64
Volume        183 non-null float64
Open          183 non-null float64
High          183 non-null float64
Low           183 non-null float64
dtypes: float64(5), object(1)
memory usage: 8.7+ KB


# Labeling :

# A) For Binary classification:
##  
## 1/for Same day approach:

In [14]:
df['SameDay_Binary']=0
for i in range(183):
    df['SameDay_Binary'][i]=int(np.sign(df['Close/Last'][i]-df['Open'][i]))

C:\Users\Hadeel\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


## 2/Previous day approach :

In [15]:
df['PreviousDay_Binary']=0
for i in range(183):
    try:
        df['PreviousDay_Binary'][i]=int(np.sign(df['Close/Last'][i]-df['Close/Last'][i-1]))
    except KeyError:
        df['PreviousDay_Binary'][i]=int(np.sign(df['Close/Last'][i]-np.mean(df['Close/Last'])))
        

C:\Users\Hadeel\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\Hadeel\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


### result :

In [16]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 183 entries, 0 to 182
Data columns (total 8 columns):
Date                  183 non-null object
Close/Last            183 non-null float64
Volume                183 non-null float64
Open                  183 non-null float64
High                  183 non-null float64
Low                   183 non-null float64
SameDay_Binary        183 non-null int64
PreviousDay_Binary    183 non-null int64
dtypes: float64(5), int64(2), object(1)
memory usage: 11.6+ KB


In [17]:
df.head()

,Date,Close/Last,Volume,Open,High,Low,SameDay_Binary,PreviousDay_Binary
0,2021-04-07,134.93,2976136.0,133.8400,134.9400,133.780,1,-1
1,2021-04-08,135.12,4087228.0,134.5700,135.6299,134.160,1,1
2,2021-04-09,135.73,3023916.0,134.8700,135.7400,134.710,1,1
3,2021-04-10,133.61,4811004.0,133.1625,134.0750,132.185,1,-1
4,2021-04-11,133.61,4811004.0,133.1625,134.0750,132.185,1,0


### Mapping labels to (0 and 1)

In [18]:
df.loc[df['SameDay_Binary'] > 0,'SameDay_Binary']=1
df.loc[df['SameDay_Binary'] <= 0,'SameDay_Binary']=0
df.loc[df['PreviousDay_Binary'] > 0,'PreviousDay_Binary']=1
df.loc[df['PreviousDay_Binary'] <= 0,'PreviousDay_Binary']=0

In [19]:
df.head()

,Date,Close/Last,Volume,Open,High,Low,SameDay_Binary,PreviousDay_Binary
0,2021-04-07,134.93,2976136.0,133.8400,134.9400,133.780,1,0
1,2021-04-08,135.12,4087228.0,134.5700,135.6299,134.160,1,1
2,2021-04-09,135.73,3023916.0,134.8700,135.7400,134.710,1,1
3,2021-04-10,133.61,4811004.0,133.1625,134.0750,132.185,1,0
4,2021-04-11,133.61,4811004.0,133.1625,134.0750,132.185,1,0


# B) Percentage Change technique:
##         
## 1)Same day approach :

In [20]:
df['SameDay_Percentage']=0.0
for i in range(183):
    df['SameDay_Percentage'][i]=((df['Close/Last'][i]-df['Open'][i])/df['Open'][i])*100

C:\Users\Hadeel\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


## 2) Previous day percentage :

In [21]:
df['PreviousDay_Percentage']=0.0
for i in range(183):
    try:
        df['PreviousDay_Percentage'][i]=(((df['Close/Last'][i]-df['Open'][i-1])/df['Open'][i-1])*100)
    except KeyError:
        df['PreviousDay_Percentage'][i]=(((df['Close/Last'][i]-np.mean(df['Open']))/np.mean(df['Open']))*100)
        

C:\Users\Hadeel\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\Hadeel\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


### retult:

In [22]:
df.head(10)

,Date,Close/Last,Volume,Open,High,Low,SameDay_Binary,PreviousDay_Binary,SameDay_Percentage,PreviousDay_Percentage
0,2021-04-07,134.93,2976136.0,133.8400,134.9400,133.780,1,0,0.814405,-3.004140
1,2021-04-08,135.12,4087228.0,134.5700,135.6299,134.160,1,1,0.408709,0.956366
2,2021-04-09,135.73,3023916.0,134.8700,135.7400,134.710,1,1,0.637651,0.862005
3,2021-04-10,133.61,4811004.0,133.1625,134.0750,132.185,1,0,0.336056,-0.934233
4,2021-04-11,133.61,4811004.0,133.1625,134.0750,132.185,1,0,0.336056,0.336056
5,2021-04-12,134.59,3753959.0,135.0200,135.3700,133.850,0,1,-0.318471,1.071998
6,2021-04-13,131.18,8033530.0,133.0000,133.6200,130.380,0,0,-1.368421,-2.844023
7,2021-04-14,132.63,5868049.0,131.3050,132.7800,130.520,1,1,1.009101,-0.278195
8,2021-04-15,132.58,3883955.0,133.2800,133.8700,132.220,0,0,-0.525210,0.971022
9,2021-04-16,133.59,5291756.0,133.0000,134.1000,132.950,1,1,0.443609,0.232593


### mapping to (0,1,-1):

In [23]:
df.loc[df['SameDay_Percentage'] > 0.5,'SameDay_Percentage']=1
df.loc[df['SameDay_Percentage'] < -0.5,'SameDay_Percentage']=-1
df['SameDay_Percentage']=np.where(((df['SameDay_Percentage'] >= -0.5))&(df['SameDay_Percentage'] <=0.5),0,df['SameDay_Percentage'])
df.loc[df['PreviousDay_Percentage'] > 0.5,'PreviousDay_Percentage']=1
df.loc[df['PreviousDay_Percentage'] < -0.5,'PreviousDay_Percentage']=-1
df['PreviousDay_Percentage']=np.where(((df['PreviousDay_Percentage'] >= -0.5))&(df['PreviousDay_Percentage'] <=0.5),0,df['PreviousDay_Percentage'])

In [24]:
df['PreviousDay_Percentage'] = df['PreviousDay_Percentage'].apply(int)
df['SameDay_Percentage'] = df['SameDay_Percentage'].apply(int)


In [25]:
df.drop([172,173,174,175,176,177,178,179,180,181,182],axis=0,inplace=True)

# Saving processed and labeled dataframe to csv file for modeling purpose :

In [26]:
df.to_csv('IBM_modeling_data.csv')